In [1]:
import sys, os, pickle

sys.path.insert(0, '/home/jupyter/AdversarialRobustness')
sys.path.insert(0, '/home/jupyter/AdversarialRobustness/vim')

# Add sbin to path for Trigon - fix of common Debian error
original_path = os.environ.get('PATH')
os.environ['PATH'] = original_path + ':/sbin'

import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import requests
from matplotlib import pyplot as plt
from torchvision import datasets
from transformers import ViTImageProcessor, ViTForImageClassification

from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

import loadVim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def save_checkpoint(state, filename="checkpoint.pth.tar"):
    torch.save(state, filename)

def load_checkpoint(filename="checkpoint.pth.tar", device=torch.device('cuda')):
    checkpoint = torch.load(filename, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch = checkpoint['epoch']
    return model, optimizer, epoch, checkpoint.get('loss', 0)

In [3]:
def compute_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1) 
    correct = torch.sum(preds == labels).item() 
    return correct / len(labels)

In [4]:
def validate(model, valid_loader, criterion, device):
    model.eval()
    total_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs.logits, labels)

            accuracy = compute_accuracy(outputs.logits, labels)
            total_accuracy += accuracy
            total_loss += loss.item()
    
    average_loss = total_loss / len(valid_loader)
    average_accuracy = total_accuracy / len(valid_loader)
    return average_loss, average_accuracy

# Training Loop

In [6]:
def train(model, train_loader, valid_loader, optimizer, criterion, device, num_epochs, pathmodel):
    logs = []
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        total_accuracy = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs.logits, labels)

            accuracy = compute_accuracy(outputs.logits, labels)
            total_accuracy += accuracy

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        
        average_loss = total_loss / len(train_loader)
        average_accuracy = total_accuracy / len(train_loader)
        val_loss, val_accuracy = validate(model, valid_loader, criterion, device)
        
        logs.append({"train":{"loss":average_loss, "acc":average_accuracy}, "val":{"loss":val_loss, "acc":val_accuracy}})

        print(f"Epoch {epoch+1}, Train Loss: {average_loss:.4f}, Train Accuracy: {average_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    save_checkpoint({'state_dict': model.vim_model.state_dict(), 'log':logs}, filename=pathmodel)

In [7]:
def trainWrap(pathtrain, pathval, pathmodel, nepochs):
    model, processor = loadVim.prepareDownstreamModel()

    transform = transforms.Compose([
        lambda image: processor(images=image, return_tensors='pt')['pixel_values'].squeeze(0)
    ])

    datasetTrain = datasets.ImageFolder(pathtrain, transform = transform)
    datasetVal = datasets.ImageFolder(pathval, transform = transform)

    loaderTrain = DataLoader(datasetTrain, batch_size=128, shuffle=True)
    loaderVal = DataLoader(datasetVal, batch_size=128, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    criterion = torch.nn.CrossEntropyLoss()
    model = model.to(device)
    
    train(model, loaderTrain, loaderVal, optimizer, criterion, device, nepochs, pathmodel)

In [8]:
pathtrain = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebTrainA"
pathval = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"
pathmodel = "/home/jupyter/AdversarialRobustness/models/VimA.pth"

trainWrap(pathtrain, pathval, pathmodel, 20)
# state_dict and log saved in dictionary at pathmodel.

Epoch 1, Train Loss: 1.9095, Train Accuracy: 0.2878, Val Loss: 1.3994, Val Accuracy: 0.4831
Epoch 2, Train Loss: 1.3237, Train Accuracy: 0.5229, Val Loss: 1.1946, Val Accuracy: 0.5733
Epoch 3, Train Loss: 1.1047, Train Accuracy: 0.5853, Val Loss: 1.1789, Val Accuracy: 0.5279
Epoch 4, Train Loss: 1.0166, Train Accuracy: 0.6400, Val Loss: 1.0358, Val Accuracy: 0.6000
Epoch 5, Train Loss: 0.8842, Train Accuracy: 0.6776, Val Loss: 1.0370, Val Accuracy: 0.5647
Epoch 6, Train Loss: 0.8506, Train Accuracy: 0.6917, Val Loss: 0.9461, Val Accuracy: 0.6748
Epoch 7, Train Loss: 0.7604, Train Accuracy: 0.7245, Val Loss: 0.9037, Val Accuracy: 0.6864
Epoch 8, Train Loss: 0.7960, Train Accuracy: 0.6949, Val Loss: 1.0110, Val Accuracy: 0.6186
Epoch 9, Train Loss: 0.7697, Train Accuracy: 0.7079, Val Loss: 0.9418, Val Accuracy: 0.6660
Epoch 10, Train Loss: 0.7355, Train Accuracy: 0.7115, Val Loss: 0.9198, Val Accuracy: 0.6702
Epoch 11, Train Loss: 0.7267, Train Accuracy: 0.7364, Val Loss: 0.9789, Val Acc

In [9]:
pathtrain = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebTrainB"
pathval = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"
pathmodel = "/home/jupyter/AdversarialRobustness/models/VimB.pth"

trainWrap(pathtrain, pathval, pathmodel, 20)

Epoch 1, Train Loss: 1.8283, Train Accuracy: 0.3146, Val Loss: 1.3761, Val Accuracy: 0.5262
Epoch 2, Train Loss: 1.2348, Train Accuracy: 0.5428, Val Loss: 1.1845, Val Accuracy: 0.5520
Epoch 3, Train Loss: 0.9911, Train Accuracy: 0.6424, Val Loss: 1.0807, Val Accuracy: 0.5804
Epoch 4, Train Loss: 0.8882, Train Accuracy: 0.6588, Val Loss: 1.0678, Val Accuracy: 0.5999
Epoch 5, Train Loss: 0.8155, Train Accuracy: 0.7176, Val Loss: 1.0476, Val Accuracy: 0.6122
Epoch 6, Train Loss: 0.7544, Train Accuracy: 0.7310, Val Loss: 0.9935, Val Accuracy: 0.6220
Epoch 7, Train Loss: 0.6427, Train Accuracy: 0.7776, Val Loss: 1.0437, Val Accuracy: 0.6252
Epoch 8, Train Loss: 0.6508, Train Accuracy: 0.7823, Val Loss: 0.9869, Val Accuracy: 0.6415
Epoch 9, Train Loss: 0.6308, Train Accuracy: 0.7908, Val Loss: 1.0071, Val Accuracy: 0.6095
Epoch 10, Train Loss: 0.5679, Train Accuracy: 0.8289, Val Loss: 1.0041, Val Accuracy: 0.6435
Epoch 11, Train Loss: 0.5906, Train Accuracy: 0.8062, Val Loss: 0.9721, Val Acc

In [10]:
# Accuracies in hindsight:

for dataset in ["A", "B"]:
    mod = torch.load(f"/home/jupyter/AdversarialRobustness/models/Vim{dataset}.pth")
    model, processor = loadVim.prepareDownstreamModel()
    model.vim_model.load_state_dict(mod["state_dict"])
    transform = transforms.Compose([
            lambda image: processor(images=image, return_tensors='pt')['pixel_values'].squeeze(0)
        ])
    datasetTest = datasets.ImageFolder("/home/jupyter/AdversarialRobustness/CelebSubset/CelebTest", transform = transform)
    loaderTrain = DataLoader(datasetTest, batch_size=128, shuffle=False)
    print("Results for dataset ", dataset)
    print(validate(model.to(device), loaderTrain, torch.nn.CrossEntropyLoss(), device))

Results for dataset  A
(0.9388849437236786, 0.6850490196078431)
Results for dataset  B
(0.9405563771724701, 0.7212775735294117)
